# Azimuthal Integral Tutorial

## Introduction

This tutorial demonstrates how to acquire an azimuthal integral profile from a multidimensional data set in pyXem. 

The data set is a 10x10x256x256 data set of a polycrystalline gold film acquired using a Medipix3 256 by 256 pixel detector. 

This functionality has been checked to run in pyxem-0.10.0 (September 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

# Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#det'> Creating a Detector Object</a>
3. <a href='#cal'> Calculation Calibration Parameters</a>
4. <a href='#ai'> Acquiring an Azimuthal Integral</a>

Import pyXem and other required libraries

In [1]:
%matplotlib tk
import pyxem as pxm
import numpy as np

# <a id='loa'></a> 1. Loading and Inspection

Load the multidimensional diffraction pattern. 

In [71]:
dp = pxm.load_hspy('ai-demo-data.hspy',
                  assign_to='electron_diffraction2d')

In [72]:
print('The file type and size is:', dp)

The file type and size is: <ElectronDiffraction2D, title: , dimensions: (10, 10|256, 256)>


In [75]:
dp.plot(vmax=1000)

In [81]:
# A distortion correction is applied
dp.apply_affine_transformation(
    np.array([[0.99978285, 0.00341758, 0.],
             [0.00341758, 0.94621262, 0.],
             [0., 0., 1.]]),keep_dtype=True)

# <a id='det'></a> 2. Creating a Detector Object

To get an azimuthal integral, it is important to characterise the detector, particularly in case it is not well represented by a flat-field approximation. To do this, we use the PyFAI Detector class. For example, here we create a Medipix256x256 Detector object from pyXem. 

In [82]:
from pyxem.detectors import Medipix256x256Detector

In [83]:
detector = Medipix256x256Detector()

Let's check what the detector looks like

In [84]:
print(detector)
print('The shape is:', detector.max_shape)
print('The mask is: ', detector.calc_mask())

Detector Medipix256x256Detector	 Spline= None	 PixelSize= 5.500e-05, 5.500e-05 m
The shape is: (256, 256)
The mask is:  None


In the case of the Medipix3 detector, it has a 55 um pixel size and 256x256 pixels which all function. 

### 2.1. Generic Detector Object

pyXem also defines a generic detector object which can be used if you're uncertain about certain detector parameters, or need a temporary fix for something.

In [85]:
from pyxem.detectors import GenericFlatDetector

In [86]:
detector2 = GenericFlatDetector(256,256)

The 256, 256 refers to the size of the detector in its two dimensions.

# <a id='cal'></a> 3. Calculating Calibration Parameters

In addition to specifying the detector, to accurately calculate the curvature of the Ewald Sphere, it is important to specify a calibration. In addition, the wavelength is specified to do that calculation.

The calibration is calculated by knowing the camera length. Alternatively, by assuming a no curvature in the detector, it is possible to calculate the camera length from an "inverse angstroms per pixel" calibration value. We suggest calibrating to a gold pattern for a calibration value and using the latter (for electron microscopy).

In [87]:
wavelength = 2.5079e-12 #for 200 kV

### 3.1 By knowing the camera length accurately

In [88]:
camera_length = 0.24 #in metres

### 3.2 By calculating the camera length from a calibration value

In [89]:
calib = 0.009197

pix_size = 55e-6 #change to 1 if using the GenericFlatDetector()
camera_length = pix_size / (wavelength * calib * 1e10)
print('Camera Length:', camera_length)

Camera Length: 0.2384549199868201


# <a id='ai'></a> 4. Acquiring an Azimuthal Integral

The azimuthal integral is then acquired by integration using the get_azimuthal_integral() method.

First, it is key to specify the origin (in pixels) as a list [x y]. If the origin moves, instead an array of origins can be passed instead.

In [90]:
origin = [127.5, 127.5] # 0 is the first pixel 

In [91]:
ai = dp.get_azimuthal_integral(origin = origin, detector_distance = camera_length, detector=detector, 
                                wavelength=wavelength, size_1d=181)
#the size1d is chosen as 181 as there are 181 effective pixels from the centre to the corner. Any number can 
#be chosen

In [45]:
#The integration can be faster through parsing different parameters. For example: 
ai = dp.get_azimuthal_integral(origin = origin, detector_distance = camera_length, detector=detector, 
                                wavelength=wavelength, size_1d=20, 
                                kwargs_for_integrate1d={'method':"splitpixel"})

Finally, let's inspect the result.

In [92]:
ai.plot()